<a href="https://colab.research.google.com/github/nrohrbach/OGD_Monitoring_BAFU/blob/main/OGD_Monitoring_BAFU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#pip install pandas

In [1]:
pip install matplotlib

## Libraries importieren

In [2]:
import requests
import json
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib inline

## Alle Packages des BAFU abfragen

In [3]:
# Abfrage aller CKAN Packages des BAFU
Packages = "https://ckan.opendata.swiss/api/3/action/organization_show?id=bundesamt-fur-umwelt-bafu&include_datasets=True"
Packages = requests.get(Packages).json()

In [4]:
# Alle Bezeichungen und Publisher extrahieren
Title = [s['title_for_slug'] for s in Packages['result']['packages']]
Maintainer = [s['maintainer'] for s in Packages['result']['packages']]
Email = [s['maintainer_email'] for s in Packages['result']['packages']]
LastModified = [s['modified'] for s in Packages['result']['packages']]

In [34]:
# Dataframe für Packages erstellen
dict = {'Publisher': Maintainer, 'Mail': Email, 'Package': Title, 'LastModified': LastModified}
dfPackages = pd.DataFrame(dict)

In [35]:
Title = Title[0:5]

In [48]:
dfPackages = dfPackages[0:10]

In [49]:
# Mit folgender BaseURL können Details aller Packages abgerufen werden
DatasetBaseURL = 'https://ckan.opendata.swiss/api/3/action/package_show?id='

In [50]:
# Alle Package-Detailsabfragen
Datasets = []

for package in Title:
    try:
        DataSet = requests.get(DatasetBaseURL + package).json()
        Maintainer = DataSet['result']['maintainer']
        Email = DataSet['result']['maintainer_email']
        LastModified = DataSet['result']['modified']
        Issued = DataSet['result']['issued']
        License = DataSet['result']['resources'][0]['license']
        Datasets.append({'Package': package, 'Publisher': Maintainer, 'Mail': Email, 'LastModified':LastModified,'Issued':Issued,'License':License})

    except:
        Datasets.append({'Package': package, 'Publisher': 'Unbekannt', 'Mail': 'Unbekannt','LastModified':'','Issued':'','License':''})


In [51]:
# Package Details als Dataframe speichern
dfPackages = pd.DataFrame(Datasets)

In [52]:
# Falls LastModified leer ist, wird der Wert aus Issued übernommen.
dfPackages['LastModified'] = dfPackages.apply(lambda row: row['Issued'] if pd.isnull(row['LastModified']) or row['LastModified'] == '' else row['LastModified'], axis=1)

In [56]:
# Lizenzstring kürzen
dfPackages['License'] = dfPackages['License'].str.split('#').str[1]

In [57]:
dfPackages

,Package,Publisher,Mail,LastModified,Issued,License
0,geochemischer-bodenatlas-der-schweiz-zink,boden@bafu.admin.ch,boden@bafu.admin.ch,2023-06-30T00:00:00,2023-06-30T00:00:00,terms_open
1,lfi-themen-nach-flachenschadenausmass-10-klassen,Schweizerisches Landesforstinventar. Eidgenöss...,lfiweb@wsl.ch,2024-10-14T17:52:40+00:00,2023-06-22T05:44:57.346000+02:00,terms_by
2,holzflussmodell-in-der-schweiz,Achim Schafer,achim.schafer@bafu.admin.ch,2023-09-05T09:36:38.387000+00:00,2023-05-24T00:00:00,terms_open
3,bodenbelastung-durch-schwermetalle,Nationale Bodenbeobachtung NABO,nabo@bafu.admin.ch,2024-08-20T11:21:56.790000+00:00,2009-12-31T00:00:00,terms_by
4,lfi-themen-nach-baumart-5-klassen,Schweizerisches Landesforstinventar. Eidgenöss...,lfiweb@wsl.ch,2024-10-14T17:45:18+00:00,2023-06-22T05:41:17+00:00,terms_by


In [61]:
# Alle Datasets aller Packages abfragen
Datasets = []

for package in Title:
    try:
        DataSet = requests.get(DatasetBaseURL + package).json()
        # Alle Bezeichungen und Publisher extrahieren
        Format = [s['format'] for s in DataSet['result']['resources']]
        PackageId = [s['package_id'] for s in DataSet['result']['resources']]
        DisplayName = [s['display_name']['de'] for s in DataSet['result']['resources']]
        Datasets.append({'Package': package, 'Format': Format,'Displayname':DisplayName})

    except:
        Datasets.append({'Package': package, 'Format': 'Unbekannt','DisplayName':''})

In [62]:
# Dataframe der Datasets erstellen und mit dem Dataframe der Packages mergen
dfDatasets = pd.DataFrame(Datasets)
dfDatasets = dfDatasets.explode('Format', ignore_index=False)

dfDatasets = pd.merge(dfDatasets, dfPackages, how='left', on=['Package'])
dfDatasets['Date'] = datetime.today().strftime("%Y-%m-%d")

In [63]:
dfDatasets

,Package,Format,Displayname,Publisher,Mail,LastModified,Issued,License,Date
0,geochemischer-bodenatlas-der-schweiz-zink,SERVICE,"[Map (Preview) Vorschau map.geo.admin.ch, STAC...",boden@bafu.admin.ch,boden@bafu.admin.ch,2023-06-30T00:00:00,2023-06-30T00:00:00,terms_open,2025-02-11
1,geochemischer-bodenatlas-der-schweiz-zink,HTML,"[Map (Preview) Vorschau map.geo.admin.ch, STAC...",boden@bafu.admin.ch,boden@bafu.admin.ch,2023-06-30T00:00:00,2023-06-30T00:00:00,terms_open,2025-02-11
2,geochemischer-bodenatlas-der-schweiz-zink,WMS,"[Map (Preview) Vorschau map.geo.admin.ch, STAC...",boden@bafu.admin.ch,boden@bafu.admin.ch,2023-06-30T00:00:00,2023-06-30T00:00:00,terms_open,2025-02-11
3,geochemischer-bodenatlas-der-schweiz-zink,WMTS,"[Map (Preview) Vorschau map.geo.admin.ch, STAC...",boden@bafu.admin.ch,boden@bafu.admin.ch,2023-06-30T00:00:00,2023-06-30T00:00:00,terms_open,2025-02-11
4,lfi-themen-nach-flachenschadenausmass-10-klassen,SPARQL,"[SPARQL Endpoint mit Vorauswahl der Daten, SPA...",Schweizerisches Landesforstinventar. Eidgenöss...,lfiweb@wsl.ch,2024-10-14T17:52:40+00:00,2023-06-22T05:44:57.346000+02:00,terms_by,2025-02-11
5,lfi-themen-nach-flachenschadenausmass-10-klassen,SPARQL,"[SPARQL Endpoint mit Vorauswahl der Daten, SPA...",Schweizerisches Landesforstinventar. Eidgenöss...,lfiweb@wsl.ch,2024-10-14T17:52:40+00:00,2023-06-22T05:44:57.346000+02:00,terms_by,2025-02-11
6,lfi-themen-nach-flachenschadenausmass-10-klassen,HTML,"[SPARQL Endpoint mit Vorauswahl der Daten, SPA...",Schweizerisches Landesforstinventar. Eidgenöss...,lfiweb@wsl.ch,2024-10-14T17:52:40+00:00,2023-06-22T05:44:57.346000+02:00,terms_by,2025-02-11
7,holzflussmodell-in-der-schweiz,SPARQL,"[SPARQL Endpoint mit Vorauswahl des Graph, vis...",Achim Schafer,achim.schafer@bafu.admin.ch,2023-09-05T09:36:38.387000+00:00,2023-05-24T00:00:00,terms_open,2025-02-11
8,holzflussmodell-in-der-schweiz,HTML,"[SPARQL Endpoint mit Vorauswahl des Graph, vis...",Achim Schafer,achim.schafer@bafu.admin.ch,2023-09-05T09:36:38.387000+00:00,2023-05-24T00:00:00,terms_open,2025-02-11
9,bodenbelastung-durch-schwermetalle,SPARQL,"[SPARQL Endpoint mit Vorauswahl des Graph, vis...",Nationale Bodenbeobachtung NABO,nabo@bafu.admin.ch,2024-08-20T11:21:56.790000+00:00,2009-12-31T00:00:00,terms_by,2025-02-11


## Daten als CSV speichern

In [ ]:
# Übersicht Packages speichern
dfPackagesCSV = dfPackages.groupby(['Mail'])['Package'].count().reset_index()
dfPackagesCSV['Date'] = datetime.today().strftime("%Y-%m-%d")
dfPackagesCSV.to_csv("data/BAFU_OGD_Monitoring_Packages.csv", header=False, index=False, mode='a')

In [ ]:
# Übersicht Datasets speichern
dfDatasetsCSV = dfDatasets.groupby(['Mail'])['Package'].count().reset_index()
dfDatasetsCSV['Date'] = datetime.today().strftime("%Y-%m-%d")
dfDatasetsCSV.to_csv("data/BAFU_OGD_Monitoring_Datasets.csv", header=False, index=False, mode='a')

In [ ]:
# Übersicht Formate speichern
dfFormats = dfDatasets.groupby(['Format'])['Mail'].count().reset_index()
dfFormats.replace("","Fehlt",inplace=True)
dfFormats['Date'] = datetime.today().strftime("%Y-%m-%d")
dfFormats.to_csv("data/BAFU_OGD_Monitoring_Formats.csv", header=False, index=False, mode='a')

In [ ]:
# Übersicht Last Update speichern
dfPackages['LastModified'] = dfPackages['LastModified'].str.slice(0,7)
dfLastUpdateCSV = dfPackages.groupby(['LastModified'])['Mail'].count().reset_index()
dfLastUpdateCSV.replace("","1999-01",inplace=True) #Fehlende Werte werden 1999 dargestellt.
dfLastUpdateCSV['LastModified'] = pd.to_datetime(dfLastUpdateCSV['LastModified'])
dfLastUpdateCSV['Date'] = datetime.today().strftime("%Y-%m-%d")
dfLastUpdateCSV.to_csv("data/BAFU_OGD_Monitoring_LastUpdate.csv", header=False, index=False, mode='a')

## Visualisierungen erstellen

In [ ]:
#Packages - Daten reinladen
dfPackages = pd.read_csv("data/BAFU_OGD_Monitoring_Packages.csv", parse_dates=['Date'])

In [ ]:
# Linechart mit allen BAFU OGD-Publikationen
dfPackagesBAFU = dfPackages.groupby(dfPackages['Date']).sum()
dfPackagesBAFU = dfPackagesBAFU.drop("Mail", axis='columns')

dfPackagesBAFU.plot(figsize=(15,10))
plt.legend(loc='upper left')
plt.title("Anzahl OGD Publikationen")
plt.savefig('plots/PackagesBAFULinechart.png')
plt.close()

In [ ]:
#Packages Barchart
dfPackagesBar = dfPackages.loc[dfPackages['Date']==datetime.today().strftime("%Y-%m-%d")]
dfPackagesBar = dfPackagesBar.sort_values('Package',ascending=False)

fig, ax = plt.subplots()
ax.bar(dfPackagesBar['Mail'], dfPackagesBar['Package'])
ax.set_title('Anzahl OGD Publikationen')
plt.xticks(rotation = 90)
plt.ylabel("Anzahl Masten")
plt.rcParams["figure.figsize"] = (10,5)
ax.bar_label(ax.containers[0], label_type='edge')
plt.savefig('plots/PackagesBarchart.png',bbox_inches='tight')
plt.close()

In [ ]:
#Packages - Linechart
dfPackagesLine = dfPackages.pivot(index="Date", columns=['Mail'],values="Package")

dfPackagesLine.plot(figsize=(15,10))
plt.legend(loc='lower left')
plt.title("Anzahl OGD Publikationen")
plt.savefig('plots/PackagesLinechart.png')
plt.close()

In [ ]:
#Datasets - Daten reinladen
dfDatasets = pd.read_csv("data/BAFU_OGD_Monitoring_Datasets.csv", parse_dates=['Date'])

In [ ]:
#Datasets Barchart
dfDatasetsBar = dfDatasets.loc[dfDatasets['Date']==datetime.today().strftime("%Y-%m-%d")]
dfDatasetsBar = dfDatasetsBar.sort_values('Package',ascending=False)

fig, ax = plt.subplots()
ax.bar(dfDatasetsBar['Mail'], dfDatasetsBar['Package'])
ax.set_title('Anzahl OGD Datensätze')
plt.xticks(rotation = 90)
plt.ylabel("Anzahl Masten")
plt.rcParams["figure.figsize"] = (10,5)
ax.bar_label(ax.containers[0], label_type='edge')
plt.savefig('plots/DatasetsBarchart.png',bbox_inches='tight')
plt.close()

In [ ]:
#Datsets - Linechart
dfDatasetsLine = dfDatasets.pivot(index="Date", columns=['Mail'],values="Package")

dfDatasetsLine.plot(figsize=(15,10))
plt.legend(loc='lower left')
plt.title("Anzahl OGD Datensätze")
plt.savefig('plots/DatasetsLinechart.png')
plt.close()

In [ ]:
#Formate - Daten reinladen
dfFormats = pd.read_csv("data/BAFU_OGD_Monitoring_Formats.csv", parse_dates=['Date'])

In [ ]:
#Formats Barchart
dfFormatsBar = dfFormats.loc[dfFormats['Date']==datetime.today().strftime("%Y-%m-%d")]
dfFormatsBar = dfFormatsBar.sort_values('Mail',ascending=False)

fig, ax = plt.subplots()
ax.bar(dfFormatsBar['Format'], dfFormatsBar['Mail'])
ax.set_title('Anzahl OGD Datensätze nach Format')
plt.xticks(rotation = 90)
plt.ylabel("Anzahl Datensätze")
plt.rcParams["figure.figsize"] = (10,5)
ax.bar_label(ax.containers[0], label_type='edge')
plt.savefig('plots/FormatssBarchart.png',bbox_inches='tight')
plt.close()

In [ ]:
#Formats - Linechart
dfFormatsLine = dfFormats.pivot(index="Date", columns=['Format'],values="Mail")

dfFormatsLine.plot(figsize=(15,10))
plt.legend(loc='lower left')
plt.title("Anzahl OGD Datensätze nach Format")
plt.savefig('plots/FormatssLinechart.png')
plt.close()

In [ ]:
#Last Update - Daten reinladen
dfLastUpdate = pd.read_csv("data/BAFU_OGD_Monitoring_LastUpdate.csv", parse_dates=['Date'])
dfLastUpdate = dfLastUpdate.loc[dfLastUpdate['Date']==datetime.today().strftime("%Y-%m-%d")]
dfLastUpdate['LastModified'] = pd.to_datetime(dfLastUpdate['LastModified'])
dfLastUpdate['Mail']=pd.to_numeric(dfLastUpdate['Mail'])

In [ ]:
#Last Update - Barchart
fig, ax = plt.subplots()
plt.rcParams["figure.figsize"] = (4,2)
plt.bar(x=dfLastUpdate['LastModified'], height=dfLastUpdate['Mail'], width=20)
ax.set_title('Anzahl Publikationen')
plt.xticks(rotation = 90)
plt.ylabel("Anzahl Publikationen")
ax.bar_label(ax.containers[0], label_type='edge')
plt.rcParams["figure.figsize"] = (10,5)
plt.savefig('plots/LastUpdate.png',bbox_inches='tight')
plt.close()